In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2


Populating the interactive namespace from numpy and matplotlib


In [4]:
import pandas as pd
import fiona

# No need to pass "layer='etc'" if there's only one layer
df = []
locs = {'type': 'FeatureCollection'}
locs['features'] = []
with fiona.open('./WastewaterOutfall.gpkg') as layer:
    for feature in layer:
        df.append({
            'id': feature['id'],
            'major_minor': feature['properties']['majorMinorStatus'],
            'city_state': f"{feature['properties']['cityName']}, {feature['properties']['stateUSPS']}",
            "lat": feature['geometry']['coordinates'][1],
            "long": feature['geometry']['coordinates'][0],
            "permit_status": feature['properties']['permitStatus'],
            "text": f"{feature['properties']['cityName']}, {feature['properties']['stateUSPS']}: {feature['properties']['majorMinorStatus']}; Permit \n{feature['properties']['permitStatus']}",            
        })
        feature['properties']['cityStateName'] = f"{feature['properties']['cityName']}, {feature['properties']['stateUSPS']}"
        locs['features'].append(feature)
df = pd.DataFrame(df)
# locs['features'] = features
#         print(feature['geometry'], feature['properties']['cityName'])

df['point_score'] = 100
df.loc[df.major_minor == 'Minor', 'point_score'] = 50

df['point_color'] = 'blue'
df.loc[df.major_minor == 'Minor', 'point_color'] = 'red'

In [5]:
df.head()

,id,major_minor,city_state,lat,long,permit_status,text,point_score,point_color
0,1,Minor,"Cambridge, MA",42.363464,-71.079669,Admin Continued,"Cambridge, MA: Minor; Permit \nAdmin Continued",50,red
1,2,Minor,"Stratford, CT",41.163690,-73.128337,Expired,"Stratford, CT: Minor; Permit \nExpired",50,red
2,3,Major,"Stamford, CT",41.045462,-73.528842,Effective,"Stamford, CT: Major; Permit \nEffective",100,blue
3,4,Minor,"Bridgewater, MA",41.995350,-70.967080,Effective,"Bridgewater, MA: Minor; Permit \nEffective",50,red
4,5,Minor,"Bath, ME",43.927253,-69.810980,Effective,"Bath, ME: Minor; Permit \nEffective",50,red


In [7]:
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go

pio.renderers.default = "browser"

fig = go.Figure(data=go.Scattergeo(
    lon = df['long'],
    lat = df['lat'],
    text = df['text'],
    locationmode="USA-states",
    marker_color=df['point_color'],
    hoverinfo='text'
    ))
fig.update_layout(
    title = 'Outfall locations',
    geo_scope='usa',
    )

fig.show()

## Desal data

## Appendix

In [102]:
import plotly.graph_objects as go

import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_february_us_airport_traffic.csv')
df['text'] = df['airport'] + '' + df['city'] + ', ' + df['state'] + '' + 'Arrivals: ' + df['cnt'].astype(str)

fig = go.Figure(data=go.Scattergeo(
        lon = df['long'],
        lat = df['lat'],
        text = df['text'],
        mode = 'markers',
        marker_color = df['cnt'],
        marker = dict(
            size = 8,
            opacity = 0.8,
            reversescale = True,
            autocolorscale = False,
            symbol = 'square',
            line = dict(
                width=1,
                color='rgba(102, 102, 102)'
            ),
            colorscale = 'Blues',
            cmin = 0,
            color = df['cnt'],
            cmax = df['cnt'].max(),
            colorbar_title="Major / Minor"        
        )))

fig.update_layout(
        title = 'Most trafficked US airports<br>(Hover for airport names)',
        geo_scope='usa',
    )
fig.show()

In [60]:
df

,iata,airport,city,state,country,lat,long,cnt,text
0,ORD,Chicago O'Hare International,Chicago,IL,USA,41.979595,-87.904464,25129,"Chicago O'Hare InternationalChicago, ILArrival..."
1,ATL,William B Hartsfield-Atlanta Intl,Atlanta,GA,USA,33.640444,-84.426944,21925,"William B Hartsfield-Atlanta IntlAtlanta, GAAr..."
2,DFW,Dallas-Fort Worth International,Dallas-Fort Worth,TX,USA,32.895951,-97.037200,20662,Dallas-Fort Worth InternationalDallas-Fort Wor...
3,PHX,Phoenix Sky Harbor International,Phoenix,AZ,USA,33.434167,-112.008056,17290,"Phoenix Sky Harbor InternationalPhoenix, AZArr..."
4,DEN,Denver Intl,Denver,CO,USA,39.858408,-104.667002,13781,"Denver IntlDenver, COArrivals: 13781"
...,...,...,...,...,...,...,...,...,...
216,EAU,Chippewa Valley Regional,Eau Claire,WI,USA,44.865257,-91.485072,48,"Chippewa Valley RegionalEau Claire, WIArrivals..."
217,DBQ,Dubuque Municipal,Dubuque,IA,USA,42.402959,-90.709167,48,"Dubuque MunicipalDubuque, IAArrivals: 48"
218,RST,Rochester International,Rochester,MN,USA,43.908826,-92.497987,37,"Rochester InternationalRochester, MNArrivals: 37"
219,UTM,Tunica Municipal Airport,Tunica,MS,USA,34.681499,-90.348816,32,"Tunica Municipal AirportTunica, MSArrivals: 32"


In [39]:
import plotly.express as px
df = px.data.gapminder().query("year == 2007")
fig = px.scatter_geo(df, locations="iso_alpha",
                     color="continent", # which column to use to set the color of markers
                     hover_name="country", # column added to hover information
                     size="pop", # size of markers
                     projection="natural earth")
fig.show()

In [33]:
# from urllib.request import urlopen
# import json
# with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
#     counties = json.load(response)

# import pandas as pd
# df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/fips-unemp-16.csv",
#                    dtype={"fips": str})

# import plotly.express as px

# fig = px.choropleth(df, geojson=counties, locations='fips', color='unemp',
#                            color_continuous_scale="Viridis",
#                            range_color=(0, 12),
#                            scope="usa",
#                            labels={'unemp':'unemployment rate'}
#                           )
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
# fig.show()

